<a href="https://colab.research.google.com/github/dewUmesh/pytorch/blob/main/LR_using_torch_inbuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 we can train the model using the following steps:

  1. Generate predictions

  2. Calculate the loss

  3. Compute gradients w.r.t the weights and biases

  4. Adjust the weights by subtracting a small quantity proportional to the gradient

  5. Reset the gradients to zero


In [1]:
import torch
import numpy as np

# **Linear regression using pytorch inbuilt fetures**

In [2]:
import torch.nn as nn

In [9]:
# input features (temp,rainfall,humidity)
input = np.array([
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]
],dtype = 'float32')

In [10]:
# input targets
targets = np.array ([
                   [56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]

],dtype = 'float32')

Convert numpy to tensor

In [11]:
input = torch.from_numpy(input)

In [12]:
targets = torch.from_numpy(targets)

Use datasets and dataloader, for executing batches

In [13]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [14]:
dataset = TensorDataset(input, targets)
print (dataset[0:5])

(tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]))


In [15]:
databatches = DataLoader(dataset,5,shuffle=True)

In [16]:
for x,y in databatches:
  print (f"x= {x}")
  print (f"y= {y}")

x= tensor([[ 68.,  97.,  70.],
        [ 68.,  96.,  71.],
        [ 87., 135.,  57.],
        [ 73.,  66.,  44.],
        [101.,  44.,  37.]])
y= tensor([[102., 120.],
        [104., 118.],
        [118., 134.],
        [ 57.,  69.],
        [ 21.,  38.]])
x= tensor([[103.,  43.,  36.],
        [ 88., 134.,  59.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.]])
y= tensor([[ 20.,  38.],
        [118., 132.],
        [ 81., 101.],
        [119., 133.],
        [ 56.,  70.]])
x= tensor([[ 91.,  87.,  65.],
        [ 92.,  87.,  64.],
        [102.,  43.,  37.],
        [ 74.,  66.,  43.],
        [ 69.,  96.,  70.]])
y= tensor([[ 80., 102.],
        [ 82., 100.],
        [ 22.,  37.],
        [ 57.,  69.],
        [103., 119.]])


1. Declare model

2. Predict

3. Get Loss

4. Optimize

In [17]:
import torch.nn as nn

In [18]:
model = nn.Linear(3,2)

In [19]:
predict = model(input)

In [20]:
print (predict)

tensor([[ 1.7116e+01,  4.6274e+00],
        [ 2.2947e+01,  2.0186e+00],
        [-3.1882e+00,  2.2007e+01],
        [ 3.9982e+01, -3.4018e-01],
        [ 1.1029e+01,  1.2670e+00],
        [ 1.8062e+01,  4.2854e+00],
        [ 2.3788e+01,  1.1319e+00],
        [-2.3634e+00,  2.1518e+01],
        [ 3.9036e+01,  1.8307e-03],
        [ 1.0924e+01,  7.2229e-01],
        [ 1.7957e+01,  3.7407e+00],
        [ 2.3893e+01,  1.6766e+00],
        [-4.0289e+00,  2.2894e+01],
        [ 4.0087e+01,  2.0450e-01],
        [ 1.0084e+01,  1.6090e+00]], grad_fn=<AddmmBackward0>)


In [21]:
print (list(model.parameters()))


[Parameter containing:
tensor([[ 0.4650, -0.4808,  0.3599],
        [ 0.0275,  0.3695, -0.5172]], requires_grad=True), Parameter containing:
tensor([-0.0916,  0.1100], requires_grad=True)]


In [24]:
loss_fn = nn.functional.mse_loss

In [25]:
loss = loss_fn(predict,targets)

In [26]:
print (loss)

tensor(7017.2544, grad_fn=<MseLossBackward0>)


In [27]:
optimize = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [28]:
print (optimize)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 1e-05
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# **Recursive model training**

In [29]:
def fit(num_epochs, model, loss_fn, opt, batches):
  # repeat for each batch
  for epoch in range(num_epochs):

    for x,y in batches:
      # 1. Predict
      prediction = model(x)
      # 2. Calculate loss
      loss = loss_fn(prediction,y)
      # 3. Compute change
      loss.backward()
      # 4. optimize
      opt.step()
      # 5. Reset grads
      opt.zero_grad()
    # print progess
    if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# **Call fit**

In [31]:
fit(1000,model,loss_fn,optimize,databatches)

Epoch [10/1000], Loss: 33.1849
Epoch [20/1000], Loss: 50.9713
Epoch [30/1000], Loss: 45.6299
Epoch [40/1000], Loss: 29.9060
Epoch [50/1000], Loss: 43.4510
Epoch [60/1000], Loss: 39.7935
Epoch [70/1000], Loss: 29.9738
Epoch [80/1000], Loss: 40.1926
Epoch [90/1000], Loss: 27.9469
Epoch [100/1000], Loss: 8.3126
Epoch [110/1000], Loss: 22.8240
Epoch [120/1000], Loss: 15.7556
Epoch [130/1000], Loss: 12.3098
Epoch [140/1000], Loss: 3.8013
Epoch [150/1000], Loss: 13.7941
Epoch [160/1000], Loss: 11.0548
Epoch [170/1000], Loss: 8.0600
Epoch [180/1000], Loss: 16.2486
Epoch [190/1000], Loss: 12.4484
Epoch [200/1000], Loss: 6.2869
Epoch [210/1000], Loss: 15.7615
Epoch [220/1000], Loss: 15.7423
Epoch [230/1000], Loss: 5.9136
Epoch [240/1000], Loss: 12.4852
Epoch [250/1000], Loss: 10.7917
Epoch [260/1000], Loss: 9.9095
Epoch [270/1000], Loss: 5.6005
Epoch [280/1000], Loss: 9.7398
Epoch [290/1000], Loss: 12.5065
Epoch [300/1000], Loss: 5.3010
Epoch [310/1000], Loss: 5.3474
Epoch [320/1000], Loss: 6.6

# **Test predictions using trained model**

In [32]:
test_predictions = model(input)
print (test_predictions)

tensor([[ 56.9740,  70.4569],
        [ 82.0959, 100.2398],
        [118.2530, 133.4261],
        [ 20.8594,  37.9265],
        [101.9104, 117.9000],
        [ 55.7283,  69.3713],
        [ 81.9536, 100.2801],
        [118.5494, 133.9968],
        [ 22.1052,  39.0120],
        [103.0138, 119.0258],
        [ 56.8317,  70.4972],
        [ 80.8502,  99.1543],
        [118.3953, 133.3858],
        [ 19.7560,  36.8006],
        [103.1561, 118.9855]], grad_fn=<AddmmBackward0>)


In [33]:
# Actual targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


In [34]:
# Difference between two matrix is
with torch.no_grad():
  print (test_predictions - targets)

tensor([[ 0.9740,  0.4569],
        [ 1.0959, -0.7602],
        [-0.7470,  0.4261],
        [-1.1406,  0.9265],
        [-1.0896, -1.1000],
        [-1.2717,  0.3713],
        [ 1.9536, -1.7199],
        [ 0.5494,  1.9968],
        [ 1.1052,  1.0120],
        [-0.9862,  1.0258],
        [-0.1683,  1.4972],
        [-1.1498, -0.8457],
        [ 0.3953, -0.6142],
        [-0.2440, -1.1994],
        [ 1.1561, -1.0145]])


# **Evaluate model**

**What would be the yield for applie and oranges for new regions with differen temp,rainfall,humidity**

In [35]:
new_prediction = model(torch.tensor([[75, 63, 44.]]))
print (new_prediction)

tensor([[53.4980, 67.5181]], grad_fn=<AddmmBackward0>)
